In [1]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [2]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: shuffled_binary_fullrank
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:07<00:00, 1335.16it/s]


{'basic': {'expected_sample_burst_size': 5.492731048805815,
           'expected_sample_data_size': 3.999848563516788,
           'expected_time_to_recover': 42.7539,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.10877793290798805,
             'bit_redundancy': 190.51609567122586,
             'ideal_bit_efficency': 0.2581555705911739,
             'ideal_bit_redundancy': 66.82175568239303,
             'ideal_packet_efficiency': 1.2907778529558693,
             'ideal_packet_redundancy': 13.364351136478605,
             'ideal_time_to_recover': 18.015032002233436,
             'ideal_transmitted_bits': 90.07516001116718,
             'packet_efficiency': 0.5438896645399403,
             'packet_redundancy': 38.10321913424517,
             'permeability': 0.4213658169718654,
             'saturation': 4.650680865754831,
             'transmitted_bits': 213.7695}}
Found implementation: shuffled_binary_matrix
  First time se

benchmark: 100%|██████████| 10000/10000 [00:07<00:00, 1369.09it/s]


{'basic': {'expected_sample_burst_size': 5.468831972020217,
           'expected_sample_data_size': 3.9978247424878974,
           'expected_time_to_recover': 43.260999999999996,
           'packet_size': 5,
           'payload_bits': 25.0},
 'derived': {'bit_efficency': 0.11557754097223828,
             'bit_redundancy': 191.30499999999998,
             'ideal_bit_efficency': 0.2736820583111113,
             'ideal_bit_redundancy': 66.34687218546476,
             'ideal_packet_efficiency': 1.3684102915555565,
             'ideal_packet_redundancy': 13.269374437092953,
             'ideal_time_to_recover': 18.269374437092953,
             'ideal_transmitted_bits': 91.34687218546476,
             'packet_efficiency': 0.5778877048611915,
             'packet_redundancy': 38.260999999999996,
             'permeability': 0.4223058745080547,
             'saturation': 5.0,
             'transmitted_bits': 216.30499999999998}}
Found implementation: single_binary_fullrank
  First time seen. B

benchmark: 100%|██████████| 10000/10000 [00:02<00:00, 3793.87it/s]


{'basic': {'expected_sample_burst_size': 5.5135231190247715,
           'expected_sample_data_size': 3.999147030600277,
           'expected_time_to_recover': 25.579000000000004,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.1818163675575601,
             'bit_redundancy': 104.64159567122587,
             'ideal_bit_efficency': 0.43248200657389907,
             'ideal_bit_redundancy': 30.513929283522145,
             'ideal_packet_efficiency': 2.1624100328694955,
             'ideal_packet_redundancy': 6.102785856704429,
             'ideal_time_to_recover': 10.75346672245926,
             'ideal_transmitted_bits': 53.7673336122963,
             'packet_efficiency': 0.9090818377878005,
             'packet_redundancy': 20.92831913424517,
             'permeability': 0.42040215498882905,
             'saturation': 4.650680865754831,
             'transmitted_bits': 127.89500000000002}}
Found implementation: single_primitive_

benchmark: 100%|██████████| 10000/10000 [00:01<00:00, 6016.05it/s]


{'basic': {'expected_sample_burst_size': 5.482057627717872,
           'expected_sample_data_size': 3.9991161392964467,
           'expected_time_to_recover': 9.2061,
           'packet_size': 5,
           'payload_bits': 2.584962500721156},
 'derived': {'bit_efficency': 0.056157602040411383,
             'bit_redundancy': 43.44553749927884,
             'ideal_bit_efficency': 0.13313941499524665,
             'ideal_bit_redundancy': 16.830493852406462,
             'ideal_packet_efficiency': 0.6656970749762333,
             'ideal_packet_redundancy': 3.366098770481292,
             'ideal_time_to_recover': 3.8830912706255236,
             'ideal_transmitted_bits': 19.41545635312762,
             'packet_efficiency': 0.2807880102020569,
             'packet_redundancy': 8.689107499855767,
             'permeability': 0.42179546937634005,
             'saturation': 0.5169925001442313,
             'transmitted_bits': 46.030499999999996}}
